In [ ]:
sub_question = '關於多發性硬化症（MS）現今常用的診斷檢查有哪些？'

# import

In [ ]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import (
    FunctionAgent,
    ReActAgent,
)

# tools

In [ ]:
# tools
from llama_index.tools.tavily_research.base import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=TAVILY_API_KEY,
)
type(tavily_tool)  # llama_index.tools.tavily_research.base.TavilyToolSpec

# 來學一下自訂 python function 當成 tool

In [ ]:
#tavily_tool_list = tavily_tool.to_tool_list()
#type(tavily_tool_list[0])  # llama_index.core.tools.function_tool.FunctionTool

from typing import List, Dict
from llama_index.core import Document
from llama_index.core.tools import FunctionTool
MAX_RESULTS = 3

def tavily_search(query: str) -> List[Document]:
    """Search Tavily and return a list[Document], one per result."""
    docs = tavily_tool.search(query=query, max_results=MAX_RESULTS)
    return docs

tavily_function_tool = FunctionTool.from_defaults(
    fn=tavily_search,
    name="tavily_search",
    description="Search Tavily and return a list of Document objects (one block per result).",
)
# 本來是直接 tavily_tool.to_tool_list() -> 參數有 query 跟 max_result
# 現在是包成 FunctionTool.from_defaults -> 參數只有 query
# call: response = tavily_function_tool('多發性硬化症')

# ReAct Agent

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context

llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False)  # streaming False for non-verified organisations
agent = ReActAgent(tools=[tavily_function_tool], llm=llm, streaming=False, verbose=False)
# Create a context to store the conversation history/session state
ctx = Context(agent)

In [ ]:
from llama_index.core.agent.workflow import AgentInput, AgentOutput, ToolCall, ToolCallResult

sub_question = '關於多發性硬化症（MS）現今常用的診斷檢查有哪些？'
handler = agent.run(sub_question, ctx=ctx)
rvs = []
async for ev in handler.stream_events(expose_internal=False):
    name = ev.__class__.__name__
    print(f"-----stream event: {name}")
    rvs.append((name, ev))

    if isinstance(ev, AgentInput):
        print(f"len of chat message: {len(ev.input)}")
    elif isinstance(ev, AgentOutput):
        print(ev.response.blocks[0].text)
    elif isinstance(ev, ToolCall):
        print(f"{ev.tool_name}: {ev.tool_kwargs}")
    elif isinstance(ev, ToolCallResult):
        num_rv = len(ev.tool_output.blocks)
        print(f"num_result: {num_rv}")
response = await handler
#print(response)

In [ ]:
func_agent = FunctionAgent(tools=[tavily_function_tool], 
                           system_prompt='你是一個網路查詢助理，會先使用網路搜索工具來才回答問題。',
                           llm=llm, streaming=False, verbose=False)
ctx = Context(func_agent)

In [ ]:
from llama_index.core.agent.workflow import AgentInput, AgentOutput, ToolCall, ToolCallResult

sub_question = '關於多發性硬化症（MS）現今常用的診斷檢查有哪些？'
handler = func_agent.run(sub_question, ctx=ctx)
rvs = []
async for ev in handler.stream_events(expose_internal=False):
    name = ev.__class__.__name__
    print(f"-----stream event: {name}")
    rvs.append((name, ev))

    if isinstance(ev, AgentInput):
        print(f"len of chat message: {len(ev.input)}")
    elif isinstance(ev, AgentOutput):
        if len(ev.response.blocks):
            print(ev.response.blocks[0].text)
        else:
            print(ev.response.additional_kwargs['tool_calls'])
    elif isinstance(ev, ToolCall):
        print(f"{ev.tool_name}: {ev.tool_kwargs}")
    elif isinstance(ev, ToolCallResult):
        num_rv = len(ev.tool_output.blocks)
        print(f"num_result: {num_rv}")
response = await handler
#print(response)

In [ ]:
rvs[1][1].response.additional_kwargs['tool_calls']

In [ ]:
rvs[1][1].response

In [ ]:
rvs[0]

In [ ]:
response.tool_calls

In [ ]:
agent = FunctionAgent(
    tools=[multiply_tool, add_tool, subtract_tool],
    llm=llm,
)

# view Prompts

In [ ]:
agent.get_prompts()

In [ ]:
print(agent.get_prompts()['react_header'])

In [ ]:
agent.get_tools()